# Задание 1.1

[Свертка списка](https://ru.wikipedia.org/wiki/%D0%A1%D0%B2%D1%91%D1%80%D1%82%D0%BA%D0%B0_%D1%81%D0%BF%D0%B8%D1%81%D0%BA%D0%B0) -  это обобщенная операция над списком, c помощью которой можно преобразовать список в единое значение. Например, рассмотрим реализации свертки слева и свертки справа (левоассоциативную свертку и правоассоциативную свертку):

In [1]:
def foldl(f, x0, lst):
    if not lst:
        return x0
    return foldl(f, f(x0, lst[0]), lst[1:])

def foldr(f, x0, lst):
    if not lst:
        return x0
    return f(lst[0], foldr(f, x0, lst[1:]))

In [2]:
#сумма
foldl(lambda x, y: x + y, 0, [1, 2, 3])

6

In [3]:
#конкатенация
foldl(lambda x, y: '{}{}'.format(x, y), '', [1, 2, 3])

'123'

Задача: реализовать foldl через foldr и наоборот. Вместо многоточий нужно вставить выражения, которые бы привели к нужному результату. Модифицировать сам список нельзя. 

In [4]:
def foldl2(f, x0, lst):
    return foldr(lambda x,y: lambda t: y(f(t,x)), lambda x: x, lst)(x0)

In [5]:
def foldr2(f, x0, lst):
    return foldl(lambda y,x: lambda t: y(f(x,t)), lambda x: x, lst)(x0)

# Задание 1.2

нужно написать функцию, которая принимает две строки и проверяет, входит ли хотя бы одна перестановка второй строки в первую. Например:

> a = 'abcrotm'
> 
> b = 'tro'

функция def check_inv(a, b) вернет True, так как 'rot' содержится в 'abcrotm'. Нужно подумать как можно более оптимальный алгоритм и оценить его сложность. 

In [6]:
def check_inv(a, b):
    sign = "qwertyuiopasdfghjklzxcvbnmQWERTYUIOPASDFGHJKLZXCVBNM,./\';[]()=+1234567890-_*&^%$#@!<>?|"
    dict_a = {l: 0 for l in (sign)}
    dict_b = {l: 0 for l in (sign)}
    for i in a[:len(b) - 1]:
        dict_a[i] += 1

    for i in b:
        dict_b[i] += 1
    
    for i in range(len(b) - 1, len(a)):
        dict_a[a[i]] += 1
        if(dict_a == dict_b):
            return True
        dict_a[a[i - len(b) + 1]] -= 1
        
    return False

In [7]:
check_inv('aaabbc', 'aba')

True

In [8]:
check_inv('aaabbc', 'aca')

False

In [9]:
check_inv('aaabbc', 'cbbb')

False

# Задание 1.3

Реализовать бинарное дерево (класс Tree), в нём методы __repr__, __str__, __iter__ (итерация только по листьям).

In [10]:
class Tree:
    def __init__(self, value=None, left=None, right=None):
        self.value = value
        self.left = left
        self.right = right
        
        

    def __iter__(self):
        if not self.left and not self.right:
            yield self.value
        if self.left:
            yield from self.left
        if self.right:
            yield from self.right
        


    def __str__(self):
        return f'Tree: {str(self.value)}'

    def __repr__(self):
        return f'Tree: \n' \
               f'value: {str(self.value)} \n' \
               f'left: {str(self.left)} \n' \
               f'right: {str(self.right)} \n'


tree = Tree(0, Tree(1, Tree(3), Tree(4)), Tree(2))

tree.__repr__

<bound method Tree.__repr__ of Tree: 
value: 0 
left: Tree: 1 
right: Tree: 2 
>

In [11]:
list(tree)

[3, 4, 2]

# Задание 1.4

Реализовать простейший калькулятор математических выражений:
- только целые числа
- **+**, **\-**, **\***, **\**
- скобки

**Можно использовать регулярные выражения**

In [12]:
signs = {'+': (1, lambda x, y: x + y), 
         '-': (1, lambda x, y: x - y),
         '*': (2, lambda x, y: x * y),
         '/': (2, lambda x, y: x / y)}

def separator(string):
    item = ''
    for s in string:
        if s in '1234567890':
            item += s
        elif item:
            yield int(item)
            item = ''
        if s in signs or s in "()":
            yield s
    if item:
        yield int(item)

def procedure(expression):
    res = []
    for item in expression:
        if item in signs:
            while res and res[-1] != "(" and signs[item][0] <= signs[res[-1]][0]:
                yield res.pop()
            res.append(item)
        elif item == ")":
            while res:
                x = res.pop()
                if x == "(":
                    break
                yield x
        elif item == "(":
            res.append(item)
        else:
            yield item
    while res:
        yield res.pop()
        
def calc(expr):
    res = []
    expression = procedure(separator(expr))
    for item in expression:
        if item in signs:
            y, x = res.pop(), res.pop()
            res.append(signs[item][1](x, y))
        else:
            res.append(item)
    return res[0]       



In [13]:
calc('2 * (15 - 3 * 4) - 2') == 4

True